<a href="https://colab.research.google.com/github/azal17/computer_networks/blob/main/IPaddress_and_gbn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Assigning IPV4 addresses to devices**


In [1]:
from math import log2, ceil

def calculate_log_values(host):
    log_value = log2(host)
    ceil_value = ceil(log_value)
    return ceil_value

def calculate_blocksize(n_bits):
    rem = n_bits % 8
    binary_str = ""

    while rem > 0:
        binary_str += "1"
        rem -= 1

    i = 8 - len(binary_str)
    while i > 0:
        binary_str += "0"
        i -= 1

    binary_str = binary_str.ljust(8, '0')

    blocksize = 0
    for i in range(8):
        blocksize += int(binary_str[i]) * (2 ** (7 - i))

    return 256 - blocksize

def calculate_subnets(hosts, ip_add):
    current_ip = ip_add
    index = ip_add.find('/')
    sub = ip_add[index + 1:]
    eff_octet = int(int(sub) / 8)
    ip_add = ip_add[:index]
    tokens = ip_add.split('.')

    # Sort hosts array in descending order
    hosts.sort(reverse=True)

    results = []
    for host in hosts:
        print(f"\nCalculating for host requirement: {host}")
        x = calculate_log_values(host)
        blocksize = calculate_blocksize(32 - x)

        nid_1 = current_ip
        tokens[eff_octet] = str(int(tokens[eff_octet]) + blocksize)
        nid_2 = '.'.join(tokens)

        results.append({
            "host": host,
            "nid_1": nid_1,
            "nid_2": nid_2,
            "allocated_ips": x,
            "blocksize": blocksize
        })

        current_ip = nid_2

    return results

ip_add = input("Enter the IP address in CIDR notation (e.g., 192.168.1.0/24): ")

hosts_input = input("Enter host requirements: ")
hosts = list(map(int, hosts_input.strip().split()))

subnet_details = calculate_subnets(hosts, ip_add)

for subnet in subnet_details:
    print(f"\nHost requirement: {subnet['host']}")
    print(f"nid_1: {subnet['nid_1']}")
    print(f"nid_2: {subnet['nid_2']}")
    print(f"Allocated IPs: {subnet['allocated_ips']}")
    print(f"Block size: {subnet['blocksize']}")


Enter the IP address in CIDR notation (e.g., 192.168.1.0/24): 192.168.1.0/24
Enter host requirements: 20 62 120

Calculating for host requirement: 120

Calculating for host requirement: 62

Calculating for host requirement: 20

Host requirement: 120
nid_1: 192.168.1.0/24
nid_2: 192.168.1.128
Allocated IPs: 7
Block size: 128

Host requirement: 62
nid_1: 192.168.1.128
nid_2: 192.168.1.192
Allocated IPs: 6
Block size: 64

Host requirement: 20
nid_1: 192.168.1.192
nid_2: 192.168.1.224
Allocated IPs: 5
Block size: 32


**selective repeat**

In [15]:
import time
import random

def send_data(window_size, data):
    ack = 0
    while ack < len(data):
        print(f"Sending packets from {ack} to {min(ack + window_size, len(data))}")
        ack = receive_ack(ack, data, window_size)

def receive_ack(ack, data, window_size):
    received_ack = ack
    start_time = time.time()
    received_packets = set()

    while time.time() - start_time < 5:
        time.sleep(1)
        if random.random() < 0.2:
            print("Packet loss, ACK not received for some packets")
            continue

        received_ack += 1
        received_packets.add(received_ack)
        print(f"Received ACK {received_ack}")

        if received_ack == ack + window_size or received_ack == len(data):
            break

    for i in range(ack + 1, min(ack + window_size, len(data)) + 1):
        if i not in received_packets:
            print(f"Packet {i} lost, resending packet {i}")

            time.sleep(1)
            print(f"Packet {i} resent")

    return received_ack

window_size = 3
data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
send_data(window_size, data)


Sending packets from 0 to 3
Received ACK 1
Received ACK 2
Received ACK 3
Sending packets from 3 to 6
Received ACK 4
Packet loss, ACK not received for some packets
Received ACK 5
Packet loss, ACK not received for some packets
Received ACK 6
Sending packets from 6 to 9
Received ACK 7
Received ACK 8
Packet loss, ACK not received for some packets
Received ACK 9
Sending packets from 9 to 10
Received ACK 10


**gbn**

In [22]:
import time
import random

def send_data(window_size, data):
    ack = 0
    while ack < len(data):
        print(f"Sending packets from {ack} to {min(ack + window_size, len(data))}")
        time.sleep(1)
        ack = receive_ack(ack, data, window_size)

def receive_ack(ack, data, window_size):
    received_ack = ack
    start_time = time.time()
    while time.time() - start_time < 5:
        time.sleep(1)
        if random.random() < 0.2:
            print("Packet dropped, ACK not received")
            continue
        received_ack += 1
        print(f"Received ACK {received_ack}")
        if received_ack == ack + window_size or received_ack == len(data):
            return received_ack
    print("Timeout, resending packets...")
    return ack


window_size = 3
data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
send_data(window_size, data)


Sending packets from 0 to 3
Received ACK 1
Packet dropped, ACK not received
Received ACK 2
Received ACK 3
Sending packets from 3 to 6
Received ACK 4
Received ACK 5
Received ACK 6
Sending packets from 6 to 9
Packet dropped, ACK not received
Packet dropped, ACK not received
Received ACK 7
Received ACK 8
Packet dropped, ACK not received
Timeout, resending packets...
Sending packets from 6 to 9
Received ACK 7
Received ACK 8
Received ACK 9
Sending packets from 9 to 10
Received ACK 10
